<a href="https://colab.research.google.com/github/LucasLessa1/challengeBain/blob/main/Desafio_Bain_%26_Company.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Pré-Processamento

In [2]:
df1 = pd.read_csv("/content/historical-database.csv")
df1.head()

,year;city_code;product_type;product;destinated_area,Unnamed: 1
0,01/01/1974;960b4f2c94a2fb2c;temporary;Others;0,NaN
1,01/01/1974;746cc42bfb8f6b62;temporary;Others;0,NaN
2,01/01/1974;6cce2bf873870afc;temporary;Others;0,NaN
3,01/01/1974;4de42e351006a2ae;temporary;Others;0,NaN
4,01/01/1974;5b6072f8f6d37acc;temporary;Others;0,NaN


In [3]:
year = []
city_code=[]
product_type=[]
product=[]
destinated_area=[]

for i in range(len(df1)):
  aux=df1.iloc[i][0]
  aux=aux.split(";")
  year.append(aux.pop(0))
  city_code.append(aux.pop(0))
  product_type.append(aux.pop(0))
  product.append(aux.pop(0))
  destinated_area.append(aux.pop(0))

columns_name = ["year","city_code","product_type","product","destinated_area"]
data = [year,city_code,product_type,product,destinated_area]
df = pd.DataFrame()
for i in range(len(columns_name)):
  df[columns_name[i]]=data[i]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73744 entries, 0 to 73743
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   year             73744 non-null  object
 1   city_code        73744 non-null  object
 2   product_type     73744 non-null  object
 3   product          73744 non-null  object
 4   destinated_area  73744 non-null  object
dtypes: object(5)
memory usage: 2.8+ MB


In [5]:
df.isna().sum()

year               0
city_code          0
product_type       0
product            0
destinated_area    0
dtype: int64

In [6]:
df.iloc[24408-2]

year                     01/01/1994
city_code          5e1e131ec84f4c87
product_type              temporary
product                         Soy
destinated_area                    
Name: 24406, dtype: object

In [7]:
(df=='').sum()

year                   0
city_code              0
product_type           0
product                0
destinated_area    39035
dtype: int64

In [8]:
(df=='0').sum()

year                  0
city_code             0
product_type          0
product               0
destinated_area    5643
dtype: int64

In [9]:
lista=[]
teste = df.destinated_area.value_counts()['']
print(teste)

39035


In [10]:
df["destinated_area"] = df['destinated_area'].replace([''], np.nan)

In [11]:
df.isna().sum()

year                   0
city_code              0
product_type           0
product                0
destinated_area    39035
dtype: int64

In [12]:
(df=='').sum()

year               0
city_code          0
product_type       0
product            0
destinated_area    0
dtype: int64

In [13]:
df.dtypes

year               object
city_code          object
product_type       object
product            object
destinated_area    object
dtype: object

In [14]:
df = df.astype({"destinated_area": float})
df.dtypes

year                object
city_code           object
product_type        object
product             object
destinated_area    float64
dtype: object

In [15]:
df.isna().sum()

year                   0
city_code              0
product_type           0
product                0
destinated_area    39035
dtype: int64

In [16]:
aux = df["destinated_area"].mean()
df["destinated_area"] = df['destinated_area'].replace([np.nan], aux)

df.isna().sum()

year               0
city_code          0
product_type       0
product            0
destinated_area    0
dtype: int64

In [17]:
df.iloc[24408-2]

year                     01/01/1994
city_code          5e1e131ec84f4c87
product_type              temporary
product                         Soy
destinated_area        14433.850903
Name: 24406, dtype: object